In [1]:
import json
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report
import os

print(os.getcwd())

with open("new_lupus/all_shots_data.json", "r") as f:
    all_data = json.load(f)


/Users/momowang/Documents/2025/LAB/code/EHRSHOT


In [3]:
print(all_data.keys())
task_data = all_data["new_lupus"]
print(task_data.keys())

for shot, folds in task_data.items():
    sample_fold = next(iter(folds.values()))
    n_train = len(sample_fold["label_values_train_k"])
    n_val = len(sample_fold["label_values_val_k"])
    print(f"\n{shot}-shot: {len(folds)} folds; train={n_train}, val={n_val}")

dict_keys(['new_lupus'])
dict_keys(['1', '2', '4', '8', '12', '16', '24', '32', '48', '64', '128', '-1'])

1-shot: 5 folds; train=2, val=2

2-shot: 5 folds; train=4, val=4

4-shot: 5 folds; train=8, val=8

8-shot: 5 folds; train=16, val=16

12-shot: 5 folds; train=24, val=24

16-shot: 5 folds; train=32, val=32

24-shot: 5 folds; train=48, val=48

32-shot: 5 folds; train=64, val=64

48-shot: 5 folds; train=96, val=96

64-shot: 5 folds; train=128, val=128

128-shot: 5 folds; train=256, val=256

-1-shot: 1 folds; train=2570, val=2225


In [4]:
import json
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
import os

### ====== File Path ======
TASK_NAME = "new_lupus"
SHOT = "8"
FOLD = "0"

ROOT = "new_lupus"  

FEATURE_PATH = "features/clmbr_features.pkl"
ID_MAP_PATH = "splits/person_id_map.csv"
LABEL_PATH = f"new_lupus/all_shots_data.json"

# ===== Feature =====
with open("features/clmbr_features.pkl", "rb") as f:
    clmbr_features = pickle.load(f)

X_matrix = np.array(clmbr_features["data_matrix"])
patient_ids = np.array(clmbr_features["patient_ids"])

features_df = pd.DataFrame(X_matrix, index=patient_ids)
features_df.index = features_df.index.astype(int)

# ===== single  fold=====
TASK_NAME = "new_lupus"
SHOT = "8"
FOLD = "0"

with open(f"{TASK_NAME}/all_shots_data.json", "r") as f:
    all_data = json.load(f)

fold_data = all_data[TASK_NAME][SHOT][FOLD]

# train_idxs is the row number of features_df.values
train_idxs = fold_data["train_idxs"]
val_idxs   = fold_data["val_idxs"]

# use train_idxs and val_idxs to index features
X_train = features_df.values[train_idxs]
X_val   = features_df.values[val_idxs]

# do not use train_idxs to index label, just use it
y_train = np.array(fold_data["label_values_train_k"])
y_val   = np.array(fold_data["label_values_val_k"])

# =====all folds folded=====
X_train_list, y_train_list = [], []
X_val_list,   y_val_list   = [], []

for fold in ["0", "1", "2", "3", "4"]:
    fold_data = all_data[TASK_NAME][SHOT][fold]
    
    train_idxs = fold_data["train_idxs"]
    val_idxs   = fold_data["val_idxs"]
    
    X_train_list.append(features_df.values[train_idxs])
    y_train_list.append(np.array(fold_data["label_values_train_k"]))
    
    X_val_list.append(features_df.values[val_idxs])
    y_val_list.append(np.array(fold_data["label_values_val_k"]))

X_train = np.concatenate(X_train_list, axis=0)
y_train = np.concatenate(y_train_list, axis=0)

X_val = np.concatenate(X_val_list, axis=0)
y_val = np.concatenate(y_val_list, axis=0)

In [5]:
print(X_train.shape)

(80, 768)


In [6]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
y_proba = model.predict_proba(X_val)[:, 1]

print("ROC-AUC:", roc_auc_score(y_val, y_proba))
print("Classification Report:\n", classification_report(y_val, y_pred))

ROC-AUC: 0.56375
Classification Report:
               precision    recall  f1-score   support

       False       0.51      0.70      0.59        40
        True       0.52      0.33      0.40        40

    accuracy                           0.51        80
   macro avg       0.51      0.51      0.49        80
weighted avg       0.51      0.51      0.49        80

